In [1]:
import pandas as pd 
import numpy as np

def create_classification_shapley2vec(modAlgo):
    dims = [5, 30] 
    modules = ['elitist', 'mirrored', 'base_sampler', 'weights_option', 'local_restart', 'step_size_adaptation'] if modAlgo == 'modCMA' else ['mutation_base','mutation_reference','mutation_n_comps','use_archive','crossover','adaptation_method','lpsr']
    df_shapley_train = pd.read_csv(f'./results/{modAlgo}/shapley_merged/shapley_train.csv', index_col=0)
    df_shapley_test = pd.read_csv(f'./results/{modAlgo}/shapley_merged/shapley_test.csv', index_col=0)
    df_grid = pd.read_csv(f'./data/raw_data/{modAlgo}_conf_grid.csv', index_col=0)

    # save classification data for each module separately
    for module in modules:
        for dim in dims:
            budgets = [50*dim, 100*dim, 300*dim, 500*dim, 1000*dim, 1500*dim] 
            for budget in budgets:
                variable = f'.*_{budget}_{dim}'           
                sub_df = df_shapley_train[df_shapley_train.index.str.contains(fr'\b{variable}\b', regex=True)]
                module_values = df_grid[module]
                df_modules = pd.DataFrame(np.array(module_values), index = sub_df.index, columns=[module])
                sub_df = sub_df.join(df_modules)  
                sub_df = sub_df.replace({np.nan: 'N'})
                sub_df.to_csv(f'./data/classification_data/{modAlgo}/{module}_dim_{dim}_budget_{budget}.csv')


# create_classification_shapley2vec('modCMA')
create_classification_shapley2vec('modDE')


In [2]:
import pandas as pd
import numpy as np
def create_classification_shapley2vec_all_modules(modAlgo):
    dims = [5, 30]
    modules = ['elitist', 'mirrored', 'base_sampler', 'weights_option', 'local_restart', 'step_size_adaptation'] if modAlgo == 'modCMA' else ['mutation_base','mutation_reference','mutation_n_comps','use_archive','crossover','adaptation_method','lpsr']
    num_conf = 324 if modAlgo == 'modCMA' else 576
    df_shapley_train = pd.read_csv(f'./results/{modAlgo}/shapley_merged/shapley_train.csv', index_col=0)
    df_shapley_test = pd.read_csv(f'./results/{modAlgo}/shapley_merged/shapley_test.csv', index_col=0)
    df_grid = pd.read_csv(f'./data/raw_data/{modAlgo}_conf_grid.csv', index_col=0)

    for dim in dims:
        budgets = [50*dim, 100*dim, 300*dim, 500*dim, 1000*dim, 1500*dim] 
        for budget in budgets:
            sub_df = df_shapley_train[df_shapley_train.index.str.endswith(f'_{budget}_{dim}')]
            sub_df.index = range(0, num_conf)
            modules_values = df_grid[modules]
            sub_df = sub_df.join(modules_values) 
            sub_df = sub_df.replace({np.nan: 'N'})
            sub_df.to_csv(f'./data/classification_data/{modAlgo}/all_dim_{dim}_budget_{budget}.csv')

# create_classification_shapley2vec_all_modules('modCMA')
create_classification_shapley2vec_all_modules('modDE')